In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
from datetime import datetime

def get_components_djia():
    """goes onto yahoo finance and returns a list of tickers in the DJIA"""
    url = 'https://finance.yahoo.com/quote/%5EDJI/components?p=%5EDJI'
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table').find_all('a')
    components = [t.text for t in table]
    return components

def get_historical_prices(ticker, trading_days_back):
    """This function takes 2 inputs, a ticker symbol and the amount of trading days to go back in time from
    today's date.  The source of the data is yahoo finance historical prices and it will return a dataframe"""
    today = int(datetime.today().timestamp())
    if trading_days_back > 99:
        rounds = trading_days_back // 99 + 1
    else:
        rounds = 1
    end_date = today
    prices = pd.DataFrame([])
    for i in range(0, rounds):
        url = 'https://finance.yahoo.com/quote/' + ticker + '/history?period1=0&period2=' + str(end_date) + '&interval=1d&filter=history&frequency=1d'
        html = requests.get(url).text
        soup = BeautifulSoup(html, 'html.parser')
        table = soup.find_all('table')
        df = pd.read_html(str(table))[0]
        df = df[~df['Open'].str.contains('ividend')]
        df['Date'] = pd.to_datetime(df['Date'])
        prices = prices.append(df)
        prices = prices.drop_duplicates()
        end_date = int(df['Date'].min().timestamp())
        time.sleep(.5)
    prices = prices.reset_index(drop=True)
    prices = prices.head(trading_days_back)
    prices.columns = ['date', 'open', 'high', 'low', 'close', 'adj_close', 'volume']
    prices.insert(0, 'ticker', ticker)
    return prices

In [3]:
get_historical_prices('AMZN', 60)

,ticker,date,open,high,low,close,adj_close,volume
0,AMZN,2019-09-27,1748.00,1749.12,1718.40,1722.13,1722.13,2315098
1,AMZN,2019-09-26,1762.79,1763.37,1731.50,1739.84,1739.84,3536800
2,AMZN,2019-09-25,1747.36,1773.00,1723.00,1768.33,1768.33,3493200
3,AMZN,2019-09-24,1790.61,1795.71,1735.55,1741.61,1741.61,4616000
4,AMZN,2019-09-23,1777.00,1792.70,1767.32,1785.30,1785.30,2922300
5,AMZN,2019-09-20,1821.71,1830.63,1780.92,1794.16,1794.16,5341900
6,AMZN,2019-09-19,1821.02,1832.57,1817.90,1821.50,1821.50,2031500
7,AMZN,2019-09-18,1817.04,1822.06,1795.50,1817.46,1817.46,2505600
8,AMZN,2019-09-17,1807.08,1823.99,1804.10,1822.55,1822.55,1982400
9,AMZN,2019-09-16,1824.02,1825.69,1800.20,1807.84,1807.84,3675500
